# Face Detection

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

## Tensorflow version 1.* utilities setting

In [2]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

## Load an face detection model

In [3]:
def load_model(model_path):
    model = tf.saved_model.load(model_path)
    model = model.signatures['serving_default']

    return model

# Get label datas
PATH_TO_LABELS = './face_detection_model/label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Load model
model_path = './face_detection_model/tf_trt_FP16'
face_detector = load_model(model_path)

# Print model's input and output
print(face_detector.inputs)
print(face_detector.output_dtypes)
print(face_detector.output_shapes)

[<tf.Tensor 'image_tensor:0' shape=(None, None, None, 3) dtype=uint8>]
{'detection_classes': tf.float32, 'num_detections': tf.float32, 'detection_boxes': tf.float32, 'raw_detection_boxes': tf.float32, 'detection_scores': tf.float32, 'raw_detection_scores': tf.float32, 'detection_multiclass_scores': tf.float32}
{'detection_classes': TensorShape(None), 'num_detections': TensorShape(None), 'detection_boxes': TensorShape(None), 'raw_detection_boxes': TensorShape([None, None, 4]), 'detection_scores': TensorShape(None), 'raw_detection_scores': TensorShape([None, None, 2]), 'detection_multiclass_scores': TensorShape(None)}


## Wrapper function : Inference

In [4]:
def run_inference_for_single_image(model, image):
    image = np.asarray(image)
    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis,...]

    # Run inference
    output_dict = model(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy() 
                    for key,value in output_dict.items()}
    output_dict['num_detections'] = num_detections

    # detection_classes should be ints.
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

    return output_dict

## Realtime Face Detection

In [5]:
# Frame's Width, Height
FRAME_WIDTH = 640
FRAME_HEIGHT = 480

# Initialize webcam feed
video = cv2.VideoCapture(0)
ret = video.set(3, FRAME_WIDTH)
ret = video.set(4, FRAME_HEIGHT)

while(True):
    ret, frame = video.read()
    output_dict = run_inference_for_single_image(face_detector, cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    vis_util.visualize_boxes_and_labels_on_image_array(
        frame,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        line_thickness=8)

    cv2.imshow('Object detector', frame)

    # Press 'esc' to quit
    if cv2.waitKey(1) == 27:
        break

# Clean up
video.release()
cv2.destroyAllWindows()